## Testing the BPASS cluster at 10^7 years age.
Here, I try to create a 10000 solar mass BPASS cluster and ResolvedCluster. **This is to demonstrate the issue I have with the mass-matching inside of the Binary_Cluster implementation.**

In [ ]:
import spisea
from spisea import evolution, synthetic
import math
# Check if the evolution class works fine
import time
import numpy as np
t1=time.time()
bps=evolution.BPASS()
iso1=synthetic.Isochrone_Binary(8.0, 0.7, 100,math.log10(1), mass_sampling=1, filepath='/g/lu/scratch/ryotainagaki/BPASS_iso_filesTimedIsolated/')

/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/pysynphot/locations.py:346: UserWarning: Extinction files not found in /g/lu/models/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/pysynphot/refs.py:125: UserWarning: No thermal tables found, no thermal calculations can be performed. No files found for /g/lu/models/cdbs/mtab/*_tmt.fits
  'no thermal calculations can be performed. ' + str(e))
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/astropy/units/quantity.py:477: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/u/ryotainagaki/Desktop/PyPopStar/spisea/evolution.py:1771: RuntimeWarning: overflow encountered in power
  (1 / cs.au) * un.m)
/opt/anaconda3/envs/astroconda/lib/python3.7/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in greater
  result = geta

Changing to T= 50000 for T= 60160 logg=8.71
Changing to logg=5.00 for T= 60160 logg=8.71
Changing to T= 50000 for T= 50085 logg=8.82
Changing to logg=5.00 for T= 50085 logg=8.82
Changing to logg=5.00 for T= 49702 logg=9.10
Changing to T= 50000 for T= 64620 logg=8.69
Changing to logg=5.00 for T= 64620 logg=8.69
Changing to T= 50000 for T= 53296 logg=8.79
Changing to logg=5.00 for T= 53296 logg=8.79
Changing to T= 50000 for T= 75621 logg=8.66
Changing to logg=5.00 for T= 75621 logg=8.66
Changing to T= 50000 for T= 53966 logg=9.24
Changing to logg=5.00 for T= 53966 logg=9.24
Changing to T= 50000 for T= 50718 logg=8.85
Changing to logg=5.00 for T= 50718 logg=8.85
Changing to T= 50000 for T=100000 logg=8.64
Changing to logg=5.00 for T=100000 logg=8.64
Changing to logg=5.00 for T= 47667 logg=8.88
Changing to logg=5.00 for T= 47530 logg=9.02
Changing to T= 50000 for T= 56329 logg=8.74
Changing to logg=5.00 for T= 56329 logg=8.74
Changing to T= 50000 for T= 51229 logg=9.29
Changing to logg=5.0

In [ ]:
iso1.primaries[np.where(iso1.primaries['mass']< 0)]

In [ ]:
iso1.secondaries[np.where(iso1.secondaries['mass'] < 0)]

In [ ]:
iso1.secondaries[np.where(iso1.singles['mass'] < 0)]

For a sanity check, we can see that the primaries, secondaries, and single stars have phasees of 5 or of 101. Note that we will have a phase of 110 in order to indicate a mystery compact remnant that BPASS provides for NEWSECMODS (secondary star models with compact primaries). This is since we haven't added neutron stars or black holes yet. Stars with phase of -99 exist, and those are the secondary stars that have already merged.

In [ ]:
iso2=synthetic.IsochronePhot(7.0, 0.7, 100, math.log10(1), mass_sampling =1 , recomp=True) # New MIST v.1 isochrone for same metallicity

In [ ]:
from spisea import imf
from spisea.imf import imf, multiplicity
from spisea import ifmr

Make the clusters corresponding to the binary star isochrone and the MISTv.1 isochrone

In [ ]:
clus_1=synthetic.Binary_Cluster(iso1, imf.IMFSalpeter1955(multiplicity=multiplicity.MultiplicityResolvedDK()), 10000, ifmr=ifmr.IFMR_Spera15())
clus_2=synthetic.ResolvedCluster(iso2, imf.IMFSalpeter1955(multiplicity=multiplicity.MultiplicityResolvedDK()), 10000, ifmr=ifmr.IFMR_Spera15())

In [ ]:
from scipy.spatial import cKDTree as KDTree
primary_mass = 3.907104476879689
secondary_mass = 2.0712811580188424
loga=0.028763045199171167
iso = iso1

In [ ]:
kd = KDTree(np.transpose(np.array([iso.primaries['mass']/primary_mass, iso.secondaries['mass']/secondary_mass, iso.secondaries['log_a']/loga])))

In [ ]:
q_results = kd.query(np.array([[1, 1, 1]]))

In [ ]:
q_results

In [ ]:
iso1.primaries[3870]

In [ ]:
iso1.secondaries[3870]

Let's make sure that I am getting just about enough star mass for my cluster. (It was a bug before I used an adjustment factor.)

In [ ]:
clus_1.star_systems['systemMass'].sum()

In [ ]:
clus_1.companions['mass']

In [ ]:
import matplotlib.pyplot as plt
plt.hist(iso1.primaries['mass'], bins=30)
plt.yscale('log')

In [ ]:
import matplotlib.pyplot as plt
plt.hist(iso1.secondaries['mass'], bins=30)
plt.yscale('log')

In [ ]:
import matplotlib.pyplot as plt
plt.hist(iso1.singles['mass'], bins=30)
plt.yscale('log')

In [ ]:
import matplotlib.pyplot as plt
plt.hist(clus_1.star_systems['systemMass'], bins=30)
plt.yscale('log')

In [ ]:
import matplotlib.pyplot as plt
plt.hist(clus_2.star_systems['systemMass'], bins=30)
plt.yscale('log')

In [ ]:
hist, bins, patch = plt.hist(clus_1.star_systems['systemMass'], bins=30, histtype='step', label='BPASS')
plt.hist(clus_2.star_systems['systemMass'], bins=bins, histtype='step', label='MISTv.1')
plt.yscale('log')
plt.legend()

In [ ]:
hist, bins, patch = plt.hist(clus_1.star_systems['systemMass'], bins=30, histtype='step', label='BPASS')
plt.hist(clus_2.star_systems['systemMass'], bins=bins, histtype='step', label='MISTv.1')
plt.yscale('log')
plt.legend()

In [ ]:
hist, bins, patch = plt.hist(clus_1.star_systems['mass'], bins=30, histtype='step', label='BPASS')
plt.hist(clus_2.star_systems['mass'], bins=bins, histtype='step', label='MISTv.1')
plt.yscale('log')
plt.legend()

In [ ]:
hist, bins, patch = plt.hist(clus_1.companions['mass'], bins=30, histtype='step', label='BPASS')
plt.hist(clus_2.companions['mass'], bins=30, histtype='step', label='MISTv.1')

plt.yscale('log')
plt.legend()

In [ ]:
# Comparison with the MIST v1 cluster
clus_2.star_systems['systemMass'].sum()

In [ ]:
import matplotlib.pyplot as plt
plt.hist(iso1.primaries['mass'], bins=30)
plt.yscale('log')

In [ ]:
import matplotlib.pyplot as plt
plt.hist(iso1.singles['mass'], bins=30)
plt.yscale('log')

In [ ]:
import matplotlib.pyplot as plt
plt.hist(iso1.secondaries['mass'], bins=30)
plt.yscale('log')

In [ ]:
import numpy as np
np.where(iso1.secondaries['mass']>iso1.primaries['mass'])

In [ ]:
np.unique(iso1.primaries[np.where(iso1.secondaries['mass']>iso1.primaries['mass'])]['source'])

In [ ]:
clus_2.star_systems['systemMass'].sum()

In [ ]:
hist, bins, patch = plt.hist(clus_1.companions['log_a'], bins=30, histtype='step', label='BPASS')
plt.hist(clus_2.companions['log_a'], bins=30, histtype='step', label='MISTv.1')
plt.legend()